# MMLigner User Tutorial


#### Developed at Volkamerlab, Charité/FU Berlin 

by Dennis Köser

# Reference

MMLigner (Collier et al., 2017) works by minimizing the ivalue of the alignment. The ivalue is based on
the Minimum Message Length framework (Wallace and Boulton, 1968; Wallace, 2005), a Bayesian framework for
statistical inductive inference. The ivalue represents the hypothetical minimum message length needed to transmit
the computed alignment losslessly (Shannon, 1948).
Using the ivalue measure, the algorithm creates crude-but-effective strucural alignments rapidly to act as seeds.
These seeds are iteratively refined over an Expectation-Maximization approach using the I-value criterion.
By substracting the ivalue from the null model, the statistical significance of the alignment can be computed. If the
difference is positive, the alignment is significant.

Collier, J.H., Allison, L., Lesk A.M., Stuckey, P.J., Garcia de la Banda , M., Konagurthu, A.S. (2017)
Statistical inference of protein structural alignments using information and compression.
Bioinformatics, 33(7), 1005-1013

Wallace,C.S. and Boulton,D.M. (1968) An information measure for classification.
Comput. J., 11, 185–194.

Wallace,C.S. (2005) Statistical and Inductive Inference Using MinimumMessage Length.
Information Science and Statistics. SpringerVerlag, New York, NY.

Shannon,C.E. (1948) A mathematical theory of communication.
Bell Syst.Tech. J., 27, 379–423.

# Introduction


## What are the  chosen structures

Since this project was developed during the SARS-CoV-2 pandemic of 2020, we chose the main protease of SARS-Cov from 2006 (2GZ9) and the main protease of SARS-Cov 2 from 2020 (5R8T) as example structures for this tutorial. SARS-Cov and SARS-Cov 2 are strains of viruses that cause severe acute respiratory syndrome (SARS). The chosen proteases are required for the maturation of SARS-Cov and SARS-Cov 2 respectively, so they make a good target for structure-based drug design of anti-SARS drugs.

## Why they have been chosen

In addition to the relevance for the pandemic of 2020, these sructures work well as an example for this tutorial because of their relation to each other resulting in quite similar but not completly similar structures. They both have a length of 306 and, if you align them with Needelman-Wunsch, show a similarity 98.69 % and identity of 96.08 %. The alignment with MMLigner results in a RMSD of 1.244 Å, a coverage of 303 and an ivalue of 29.559 bits.

# Theory

## About the RCSB

The [RCSB PDB](http://www.rcsb.org/) (Research Collaboratory for Structural Bioinformatics Protein Data Bank) provide a global PDB archive, and makes PDB data available at no charge to all data consumers without limiations on usage.

## RMSD

The RMSD is the average distance between the atoms of tuperposed structures in Angstrom.

## coverage

The coverage of the aligned structures

## ivalue

The minimum message length of the compressed alignment in bites.

# Preperation

## How to get the structure from the CLI

To get the structures directly from the RCSB, the syntax looks like this:

In [ ]:
!structuralalignment --method=mmligner 2GZ9 5R8T

When you want to use structures which are locally saved, do this:

In [ ]:
!structuralalignment --method=mmligner PATH_OF_2GZ9 PATH_OF_5R8T

## Getting the structure in python

The method will use atomium.models as input.

If you want to get the structures from the RCSB, you can do the following:

In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

In [5]:
import atomium

structure1 = atomium.fetch("2GZ9").model
structure2 = atomium.fetch("5R8T").model

# Alignment 

Using MMligner in Python looks like this:

In [106]:
from structuralalignment.superposition.mmligner import MMLignerAligner

mmligner = MMLignerAligner()
results = mmligner.calculate([structure1, structure2])

/tmp/tmp8zrlttud
MMLigner (v1.0.2): Minimum Message Length (MML) alignment of protein structures 
   Authors:    Collier, Allison, Lesk, Stuckey, Garcia de la Banda, Konagurthu. 
   Reference:  Bioinformatics. 33(7):1005-1013, 2017.                           

Parsing Brookhaven PDB      .../structure1.pdb     nResi = 306   [ OK ]
Parsing Brookhaven PDB      .../structure2.pdb     nResi = 304   [ OK ]
Computing MMLigner alignment...
...building MFP Library                                          [ OK ]
...filtering MFP library                                      0.179 4.228 4.393 4.426 4.457 4.460 4.817 10.42310.65212.98613.08213.59014.97914.98014.99115.20015.81816.30416.33417.14018.20121.81921.91423.40324.47832.22838.56338.56738.56838.60339.31139.31739.32239.34739.34942.86042.95842.98843.99643.99844.45144.46944.48844.49344.49645.88445.90945.91045.91945.96845.97747.01347.04851.20851.24951.31452.35452.37852.38554.09658.31258.56364.41564.43466.32168.71770.17670.40570.46074.26874.65576.

In [94]:
results["scores"]["rmsd"]

1.244

In addition you can compute the ivalue of a already computed alignment. This looks like: 

In [ ]:
mmligner = MMLignerAligner()

results = mmligner.ivalue([structure1, structure2], alignment)

The results are returned as a dictionary containing the superposed structure, the RMSD, ivalue and coverage, as well as the alignment. The dictionary looks like:

# Analysis

### NGLview

If you have trouble with NGLview, follow this [troubleshooting guide](https://github.com/SBRG/ssbio/wiki/Troubleshooting#tips-for-nglview).

In [45]:
def atomium_to_nglview(*models):
    """
    Represent Atomium models in NGLView
    Parameters
    ----------
    models : atomium.Model
    Returns
    -------
    nglview.NGLWidget
    """
    import nglview as nv
    from structuralalignment.utils import enter_temp_directory

    v = nv.NGLWidget()
    with enter_temp_directory():
        for i, model in enumerate(models):
            fn = f"tmp{i}.pdb"
            model.save(fn)
            fs = nv.FileStructure(fn)
            v.add_component(fs)
    return v

In [107]:
import nglview as nv
print("nglview version = {}".format(nv.__version__))
# your nglview version should be 1.1.7 or later

v = atomium_to_nglview(*results["superposed"])
v

nglview version = 2.7.5


NGLWidget()

In [108]:
vv = nv.show_file("/tmp/tmp8zrlttud/structure1.pdb")
vv.add_component("/tmp/tmp8zrlttud/p_superposed__1.pdb")
vv

NGLWidget()

## Report

* RMSD before and after
* coverage
* what residues are mapped